## File style conversion

Use this notebook to convert the MPI-styled headers to the new JSON format.\\

In [1]:
%load_ext autoreload
%autoreload 2

import re
from ixo.file import getFiles
from tweezers.io import TxtMpiSource
from tweezers import TweezersData

In [2]:
# Where are the original files?
inputPath = '../tweezers/data/man_data/'
# Where should we store the converted ones?
outputPath = '../tweezers/data/man_data/JSON/'

In [3]:
# get all files
files = getFiles(inputPath, suffix='.txt', recursive=True)

# sort them
dataFiles = []
tsFiles = []
psdFiles = []
for file in files:
    if file.name[0].isdigit():
        dataFiles.append(file)
    elif file.name.startswith('TS_'):
        tsFiles.append(file)
    elif file.name.startswith('PSD_'):
        psdFiles.append(file)

# find matching files in psd and ts
tsRegex = re.compile('^TS_(?P<name>.*?)\.txt$')
psdRegex = re.compile('^PSD_(?P<name>.*?)\.txt$')
dataRegex = re.compile('^(?P<name>.*?)(_zoom.*)?\.txt$')
matchedFiles = []

for dataFile in dataFiles:
    dataName = dataRegex.match(dataFile.name).groupdict()['name']
    tsFileFound = None
    psdFileFound = None
    
    for tsFile in tsFiles:
        tsName = tsRegex.match(tsFile.name).groupdict()['name']
        if tsName == dataName and tsFile.parents[1] == dataFile.parents[1]:
            tsFileFound = tsFile
            
    for psdFile in psdFiles:
        psdName = psdRegex.match(psdFile.name).groupdict()['name']
        if psdName == dataName and psdFile.parents[1] == dataFile.parents[1]:
            psdFileFound = psdFile
    
    matchedFiles.append([dataFile, tsFileFound, psdFileFound])

In [4]:
for files in matchedFiles:
    print('Processing: ' + str(files[0]))
    source = TxtMpiSource(data=files[0], ts=files[1], psd=files[2])
    # skip files that already have the JSON header
    if source.data.isJson:
        continue
    data = TweezersData(source)
    source.save(data, path=outputPath)

INFO:root:Reading timeseries from data source.
INFO:root:Reading PSD from data source.
INFO:root:Reading data from data source.
INFO:root:Reading metadata from data source.


Processing: ../tweezers/data/man_data/35_Date_2014_08_18_15_55_29.txt
Processing: ../tweezers/data/man_data/JSON/man_data/35_Date_2014_08_18_15_55_29.txt
